# Install

In [1]:
!pip3 install datasets==2.20.0 evaluate==0.4.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 718.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. T

# 5.3.2 Causal LM

### 모델

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM


model_name = "skt/kogpt2-base-v2"
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    bos_token="</s>",
    eos_token="</s>",
    unk_token="<unk>",
    pad_token="<pad>",
    mask_token="<mask>"
)
model = AutoModelForCausalLM.from_pretrained(model_name)
model

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

### 데이터셋

In [3]:
from datasets import load_dataset

split_dict = {
    "train": "train[:8000]",
    "test": "train[8000:10000]",
    "unused": "train[10000:]",
}
dataset = load_dataset("heegyu/kowikitext", split=split_dict)
del dataset["unused"]
dataset

The repository for heegyu/kowikitext contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/heegyu/kowikitext.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'revid', 'url', 'title', 'text'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['id', 'revid', 'url', 'title', 'text'],
        num_rows: 2000
    })
})

In [4]:
tokenized_dataset = dataset.map(
    lambda batch: tokenizer([f"{ti}\n{te}" for ti, te in zip(batch["title"], batch["text"])]),
    batched=True,
    num_proc=2,
    remove_columns=dataset["train"].column_names,
)
tokenized_dataset

Map (num_proc=2):   0%|          | 0/8000 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 2000
    })
})

In [5]:
max_length = 512
def group_texts(batched_sample):
    sample = {k: v[0] for k, v in batched_sample.items()}

    if sample["input_ids"][-1] != tokenizer.eos_token_id:
        for k in sample.keys():
            sample[k].append(
                tokenizer.eos_token_id if k == "input_ids" else sample[k][-1]
            )

    result = {k: [v[i: i + max_length] for i in range(0, len(v), max_length)] for k, v in sample.items()}
    return result

grouped_dataset = tokenized_dataset.map(
    group_texts,
    batched=True,
    batch_size=1,
    num_proc=2,
)
print(len(grouped_dataset["train"][0]["input_ids"]))
print(grouped_dataset)

Map (num_proc=2):   0%|          | 0/8000 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/2000 [00:00<?, ? examples/s]

512
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 18365
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 2400
    })
})


In [6]:
from transformers import DataCollatorForLanguageModeling

collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
sample = collator([grouped_dataset["train"][i] for i in range(1)])

### 문장 생성

In [7]:
inputs = tokenizer("지난해 7월, ", return_tensors="pt").to(model.device)

outputs = model.generate(inputs.input_ids, max_new_tokens=100)
result = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(result[0])

지난해 7월, 롯데백화점 본점 지하 1층 식품매장에서 판매된 '롯데 햄버거' 제품에서 대장균이 검출돼 판매 중단된 바 있다.
롯데백화점 측은 "햄버거 판매 중단은 롯데백화점 본점 식품매장의 위생과 안전관리에 대한 고객들의 신뢰가 크게 훼손된 데 따른 것"이라며 "롯데백화점 본점 식품매장은 햄버거 판매 중단을 즉각 중단하고, 롯데백화점 본점 식품


In [8]:
import torch

input_ids = tokenizer("지난해 7월, ", return_tensors="pt").to(model.device).input_ids

with torch.no_grad():
    for _ in range(100):
        next_token = model(input_ids).logits[0, -1:].argmax(-1)
        input_ids = torch.cat((input_ids[0], next_token), -1).unsqueeze(0)

print(tokenizer.decode(input_ids[0].tolist()))

지난해 7월, 롯데백화점 본점 지하 1층 식품매장에서 판매된 '롯데 햄버거' 제품에서 대장균이 검출돼 판매 중단된 바 있다.
롯데백화점 측은 "햄버거 판매 중단은 롯데백화점 본점 식품매장의 위생과 안전관리에 대한 고객들의 신뢰가 크게 훼손된 데 따른 것"이라며 "롯데백화점 본점 식품매장은 햄버거 판매 중단을 즉각 중단하고, 롯데백화점 본점 식품


# 5.3.3 Question Ansering

### 모델

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering


model_name = "skt/kogpt2-base-v2"
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    bos_token="</s>",
    eos_token="</s>",
    unk_token="<unk>",
    pad_token="<pad>",
    mask_token="<mask>"
)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
model

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of GPT2ForQuestionAnswering were not initialized from the model checkpoint at skt/kogpt2-base-v2 and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably

GPT2ForQuestionAnswering(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (qa_outputs): Linear(in_features=768, out_features=2, bias=True)
)

### 데이터셋

In [2]:
from datasets import load_dataset

dataset = load_dataset("klue", "mrc")

def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=512,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        try:
            while idx < len(sequence_ids) and sequence_ids[idx] != 1:
                idx += 1
            context_start = idx
            while idx < len(sequence_ids) and sequence_ids[idx] == 1:
                idx += 1
            context_end = idx - 1
        except Exception as e:
            print(sequence_ids, idx)
            raise e

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

tokenized_dataset = dataset.map(
    preprocess_function, 
    batched=True, 
    remove_columns=dataset["train"].column_names
)

Map:   0%|          | 0/5841 [00:00<?, ? examples/s]

In [3]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()
batch = data_collator([tokenized_dataset["train"][i] for i in range(10)])
batch

{'input_ids': tensor([[ 9233, 20493,  9032,  ...,     3,     3,     3],
         [28567, 15263, 15755,  ..., 40714,  9045, 12446],
         [21066,  8745,  9462,  ..., 12446, 42608,  8563],
         ...,
         [10528,   425, 10355,  ...,     3,     3,     3],
         [ 9150,  8160,  7109,  ...,     3,     3,     3],
         [ 9751,  9134, 35894,  ...,     3,     3,     3]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'start_positions': tensor([215,  24,   0,  52,  58,  66, 200, 311, 297, 279]),
 'end_positions': tensor([217,  26,   0,  53,  63,  71, 208, 317, 302, 283])}

### 문장 생성

In [4]:
with torch.no_grad():
  outputs = model(**batch)

answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()

predict_answer_tokens = batch["input_ids"][0, answer_start_index : answer_end_index + 1]
tokenizer.decode(predict_answer_tokens)

''

# 5.3.4 Sequence Classification

## 모델

In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "skt/kogpt2-base-v2"
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    bos_token="</s>",
    eos_token="</s>",
    unk_token="<unk>",
    pad_token="<pad>",
    mask_token="<mask>"
)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
model

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at skt/kogpt2-base-v2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=2, bias=False)
)

## 데이터셋

In [6]:
from datasets import load_dataset

dataset = load_dataset("klue", "sts")

def process_data(batch):
  result = tokenizer(batch["sentence1"], text_pair=batch["sentence2"])
  result["labels"] = [x["binary-label"] for x in batch["labels"]]
  return result

dataset = dataset.map(process_data, batched=True, remove_columns=dataset["train"].column_names)

Generating train split:   0%|          | 0/11668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/519 [00:00<?, ? examples/s]

Map:   0%|          | 0/11668 [00:00<?, ? examples/s]

Map:   0%|          | 0/519 [00:00<?, ? examples/s]

In [7]:
from transformers import DataCollatorWithPadding

collator = DataCollatorWithPadding(tokenizer)
batch = collator([dataset["train"][i] for i in range(4)])

### 문장 생성

In [8]:
with torch.no_grad():
  logits = model(**batch).logits

logits

tensor([[ 1.2998,  0.6028],
        [ 1.1638,  0.7979],
        [ 0.4034, -0.7109],
        [ 1.0960, -0.0611]])

### 평가 지표

In [9]:
import evaluate

f1 = evaluate.load("f1")
f1.compute(predictions=logits.argmax(-1), references=batch["labels"], average="micro")

{'f1': 0.75}